In [1]:
%load_ext autoreload
%aimport -tf
%aimport data
%aimport model
%autoreload 1

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import data
import model

In [3]:
tf.logging.set_verbosity(tf.logging.INFO)

In [4]:
ds = data.load_reddit_data(
    max_document_length=40,
    min_frequency=50,
    test_size=5000)

In [6]:
data.print_dataset_stats(ds)

Vocabulary Size: 12400
Train Data Shape: (385367, 40)
Dev Data Shape (5000, 40)


In [7]:
train_input_fn, dev_input_fn = data.create_train_dev_input_fns(ds, batch_size=16)

In [8]:
def simple_rnn(sequences, sequence_lengths):
    cell = tf.nn.rnn_cell.GRUCell(64)
    sequence_list = tf.unpack(sequences, axis=1)
    return tf.nn.rnn(cell, sequence_list, dtype=tf.float32, sequence_length=sequence_lengths)

model_fn = model.create_lm(
    vocab_size=len(ds.vocab.vocabulary_),
    embedding_dim=64,
    rnn_fn=simple_rnn)

In [11]:
# Monitor for sampling sentences from the estimator
sample_mon = model.SentenceSampleMonitor(
    vocab=ds.vocab,
    every_n_steps=1000,
    first_n_steps=-1)

# Monitor for development set loss
dev_monitor = tf.contrib.learn.monitors.ValidationMonitor(
    input_fn=dev_input_fn,
    every_n_steps=1000)

estimator = tf.contrib.learn.Estimator(model_fn=model_fn)

TypeError: __init__() got an unexpected keyword argument 'vocab'

In [ ]:
estimator.fit(input_fn=train_input_fn, steps=500, monitors=[sample_mon, dev_monitor])